## **PREDICCION DE ENFERMEDAD CARDIACA**


Uso del  Heart Disease Data Set para ejercicio de prediccion


**IMPORTACION LIBRERIAS Y DATASET**

In [20]:
import pandas as pd 
from pandas import read_csv
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
names = ["EDAD", "SEXO", "TD", "PSD", "COL","AZA", "ELE", "FCO","AIE", "DST","PEN", "VAS", "TAL", "RES"]
dataset = pd.read_csv(url, names=names)
dataset.head(5)

,EDAD,SEXO,TD,PSD,COL,AZA,ELE,FCO,AIE,DST,PEN,VAS,TAL,RES
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [21]:
dataset.dtypes

EDAD    float64
SEXO    float64
TD      float64
PSD     float64
COL     float64
AZA     float64
ELE     float64
FCO     float64
AIE     float64
DST     float64
PEN     float64
VAS      object
TAL      object
RES       int64
dtype: object

In [12]:
dataset.shape

(303, 14)

In [22]:
cols = dataset.columns[dataset.dtypes.eq(object)]
cols


Index(['VAS', 'TAL'], dtype='object')

In [24]:
dataset[cols] = dataset[cols].apply(pd.to_numeric, errors='coerce')
dataset.dtypes

EDAD    float64
SEXO    float64
TD      float64
PSD     float64
COL     float64
AZA     float64
ELE     float64
FCO     float64
AIE     float64
DST     float64
PEN     float64
VAS     float64
TAL     float64
RES       int64
dtype: object

In [25]:
dataset.isnull().values.any()

True

In [26]:
dataset.isnull().values.sum()

6

In [27]:
dataset = dataset.dropna()

In [28]:
dataset.isnull().values.any()

False

In [31]:
dataset.shape

(297, 14)